In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[4]').appName('Newyork-taxi-count').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 14:10:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/09 14:10:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
trips_data = spark.read.csv(f'file:///Users/lifelog/_PML/fc_spark/data/fhvhv_tripdata_2020-03.csv', header=True, inferSchema=True)
lookup_data = spark.read.csv(f'file:///Users/lifelog/_PML/fc_spark/data/taxi+_zone_lookup.csv', header=True, inferSchema=True)

lookup_data.createOrReplaceTempView('locations')
trips_data.createOrReplaceTempView('trips')

In [11]:
print(type(lookup_data))
display(lookup_data.show())
display(trips_data.show())

<class 'pyspark.sql.dataframe.DataFrame'>
+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park C

None

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

23/11/09 14:15:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
 Schema: _c0, hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
Expected: _c0 but found: 
CSV file: file:///Users/lifelog/_PML/fc_spark/data/fhvhv_tripdata_2020-03.csv


None

In [12]:
spark.sql('''
        show tables
          ''').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |locations|       true|
|         |    trips|       true|
+---------+---------+-----------+



In [15]:
# 승차 수
spark.sql('''
        SELECT locations.Borough, count(*) as Counts
        FROM trips join locations
        on trips.PULocationID = locations.LocationID
        group by locations.Borough
        Limit 10
          ''').show()

+-------------+-------+
|      Borough| Counts|
+-------------+-------+
|       Queens|2437394|
|          EWR|    362|
|      Unknown|    845|
|     Brooklyn|3735765|
|Staten Island| 178818|
|    Manhattan|4953147|
|        Bronx|2086597|
+-------------+-------+



In [17]:
# 하차 수
spark.sql('''
        SELECT locations.Borough, count(*) as Counts
        FROM trips join locations
        on trips.DOLocationID = locations.LocationID
        group by locations.Borough
        Limit 10
          ''').show()

+-------------+-------+
|      Borough| Counts|
+-------------+-------+
|       Queens|2468416|
|          EWR|  65066|
|      Unknown| 387760|
|     Brooklyn|3696684|
|Staten Island| 177727|
|    Manhattan|4553783|
|        Bronx|2043492|
+-------------+-------+



In [18]:
# 하차 수
spark.sql('''
        SELECT locations.Borough, count(*) as Counts
        FROM trips join locations
        on trips.DOLocationID = locations.LocationID
        group by locations.Borough
        Limit 10
          ''').explain(True)

== Parsed Logical Plan ==
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Aggregate ['locations.Borough], ['locations.Borough, 'count(1) AS Counts#719]
      +- 'Join Inner, ('trips.DOLocationID = 'locations.LocationID)
         :- 'UnresolvedRelation [trips], [], false
         +- 'UnresolvedRelation [locations], [], false

== Analyzed Logical Plan ==
Borough: string, Counts: bigint
GlobalLimit 10
+- LocalLimit 10
   +- Aggregate [Borough#303], [Borough#303, count(1) AS Counts#719L]
      +- Join Inner, (DOLocationID#244 = LocationID#302)
         :- SubqueryAlias trips
         :  +- View (`trips`, [_c0#235,hvfhs_license_num#236,dispatching_base_num#237,originating_base_num#238,request_datetime#239,on_scene_datetime#240,pickup_datetime#241,dropoff_datetime#242,PULocationID#243,DOLocationID#244,trip_miles#245,trip_time#246,base_passenger_fare#247,tolls#248,bcf#249,sales_tax#250,congestion_surcharge#251,airport_fee#252,tips#253,driver_pay#254,shared_request_flag#255,shared_match_flag#256,acce